#Check in on the data

In [ ]:
import pandas as pd

# Load the dataset (assuming it's in a CSV format, adjust as necessary)
file_path = '/content/training_data_clean.csv'  # Update this path to the actual location
data = pd.read_csv(file_path)

# Display basic information about the dataset
print("Dataset Overview:")
print(data.head())

# Check for missing values
print("\nMissing Values:")
print(data.isnull().sum())

# Get the distribution of the secondary structure labels
print("\nSecondary Structure Distribution:")
print(data['sst3'].value_counts())


#Random Forest

In [ ]:
#As A Function

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

def train_and_evaluate_rf(
    train_file_path,
    test_file_path,
    report_file_path,
    predictions_file_path,
    n_estimators=100,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1
):
    # Load the datasets
    train_data = pd.read_csv(train_file_path)
    test_data = pd.read_csv(test_file_path)

    # List of amino acids (for one-hot encoding)
    amino_acids = 'ACDEFGHIKLMNPQRSTVWY'

    # Function to one-hot encode a sequence
    def one_hot_encode(seq, aa_list):
        encoding = np.zeros((len(seq), len(aa_list)), dtype=int)
        for i, aa in enumerate(seq):
            if aa in aa_list:
                encoding[i, aa_list.index(aa)] = 1
        return encoding

    # Encode the sequences for training and testing
    train_encoded = [one_hot_encode(seq, amino_acids) for seq in train_data['seq']]
    test_encoded = [one_hot_encode(seq, amino_acids) for seq in test_data['seq']]

    # Find the maximum sequence length in the training and testing datasets
    max_seq_len = max(max(len(seq) for seq in train_data['seq']),
                      max(len(seq) for seq in test_data['seq']))

    # Pad the sequences to the maximum length
    train_sequences = pad_sequences(train_encoded, maxlen=max_seq_len, padding='post', dtype='float32')
    test_sequences = pad_sequences(test_encoded, maxlen=max_seq_len, padding='post', dtype='float32')

    # Encode the secondary structures as target labels
    sst3_mapping = {'H': 0, 'E': 1, 'C': 2}
    train_labels = pad_sequences([[sst3_mapping[ss] for ss in sst] for sst in train_data['sst3']],
                                 maxlen=max_seq_len, padding='post', value=-1)
    test_labels = pad_sequences([[sst3_mapping[ss] for ss in sst] for sst in test_data['sst3']],
                                maxlen=max_seq_len, padding='post', value=-1)

    # Flatten the sequences and labels
    train_sequences_flat = train_sequences.reshape(-1, train_sequences.shape[2])
    test_sequences_flat = test_sequences.reshape(-1, test_sequences.shape[2])
    train_labels_flat = train_labels.flatten()
    test_labels_flat = test_labels.flatten()

    # Create mask to filter out padded positions (-1)
    train_mask = train_labels_flat != -1
    test_mask = test_labels_flat != -1

    # Apply the mask to filter out padding
    train_sequences_flat = train_sequences_flat[train_mask]
    train_labels_flat = train_labels_flat[train_mask]
    test_sequences_flat = test_sequences_flat[test_mask]
    test_labels_flat = test_labels_flat[test_mask]

    # Train the Random Forest model with the original settings
    rf_model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42
    )
    rf_model.fit(train_sequences_flat, train_labels_flat)

    # Make predictions on the test set
    test_predictions = rf_model.predict(test_sequences_flat)

    # Generate the classification report
    report = classification_report(test_labels_flat, test_predictions, target_names=['H', 'E', 'C'])

    # Save the classification report to a text file
    with open(report_file_path, 'w') as f:
        f.write(f"Random Forest Test Accuracy: {rf_model.score(test_sequences_flat, test_labels_flat):.4f}\n\n")
        f.write("Classification Report:\n")
        f.write(report)

    # Save the predictions along with the true labels to a CSV file
    results_df = pd.DataFrame({
        'True_Label': test_labels_flat,
        'Predicted_Label': test_predictions
    })
    results_df.to_csv(predictions_file_path, index=False)

    print(f"Classification report saved to {report_file_path}")
    print(f"Predictions saved to {predictions_file_path}")

In [ ]:
train_and_evaluate_rf(
    train_file_path='/content/training_data_clean.csv',
    test_file_path='/content/test_data_clean.csv',
    report_file_path='RF_Initial_report.txt',
    predictions_file_path='RF_initial_predictions.csv'
)

#CNN

In [ ]:
#As a function
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dropout, TimeDistributed, Dense, Input
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences

def train_and_evaluate_cnn(
    train_file_path,
    test_file_path,
    report_file_path,
    predictions_file_path,
    epochs=10,
    batch_size=32
):
    # Load the datasets
    train_data = pd.read_csv(train_file_path)
    test_data = pd.read_csv(test_file_path)

    # List of amino acids (for one-hot encoding)
    amino_acids = 'ACDEFGHIKLMNPQRSTVWY'

    # Function to one-hot encode a sequence
    def one_hot_encode(seq, aa_list):
        encoding = np.zeros((len(seq), len(aa_list)), dtype=int)
        for i, aa in enumerate(seq):
            if aa in aa_list:
                encoding[i, aa_list.index(aa)] = 1
        return encoding

    # Encode the sequences for training and testing
    train_encoded = [one_hot_encode(seq, amino_acids) for seq in train_data['seq']]
    test_encoded = [one_hot_encode(seq, amino_acids) for seq in test_data['seq']]

    # Find the maximum sequence length in the training and testing datasets
    max_seq_len = max(max(len(seq) for seq in train_data['seq']),
                      max(len(seq) for seq in test_data['seq']))

    # Pad the sequences to the maximum length
    train_sequences = pad_sequences(train_encoded, maxlen=max_seq_len, padding='post', dtype='float32')
    test_sequences = pad_sequences(test_encoded, maxlen=max_seq_len, padding='post', dtype='float32')

    # Encode the secondary structures as target labels
    sst3_mapping = {'H': 0, 'E': 1, 'C': 2}
    train_labels = pad_sequences([[sst3_mapping[ss] for ss in sst] for sst in train_data['sst3']],
                                 maxlen=max_seq_len, padding='post', value=-1)
    test_labels = pad_sequences([[sst3_mapping[ss] for ss in sst] for sst in test_data['sst3']],
                                maxlen=max_seq_len, padding='post', value=-1)

    # One-hot encode the labels
    train_labels_categorical = to_categorical(train_labels, num_classes=3)
    test_labels_categorical = to_categorical(test_labels, num_classes=3)

    # Define the CNN architecture
    model = Sequential([
        Input(shape=(train_sequences.shape[1], train_sequences.shape[2])),
        Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'),
        Dropout(0.2),
        Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'),
        Dropout(0.3),
        TimeDistributed(Dense(128, activation='relu')),
        TimeDistributed(Dense(3, activation='softmax')),
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(
        train_sequences,
        train_labels_categorical,
        epochs=epochs,
        batch_size=batch_size,
        validation_split=0.1,
        verbose=1
    )

    # Make predictions on the test set
    test_predictions = model.predict(test_sequences)
    test_predictions_labels = np.argmax(test_predictions, axis=-1)
    test_true_labels = np.argmax(test_labels_categorical, axis=-1)

    # Flatten the arrays to create a single list of predictions and true labels
    test_predictions_flat = test_predictions_labels.flatten()
    test_true_labels_flat = test_true_labels.flatten()

    # Generate the classification report
    report = classification_report(test_true_labels_flat, test_predictions_flat, target_names=['H', 'E', 'C'])

    # Save the classification report to a text file
    with open(report_file_path, 'w') as f:
        f.write(f"CNN Test Accuracy: {model.evaluate(test_sequences, test_labels_categorical, verbose=0)[1]:.4f}\n\n")
        f.write("Classification Report:\n")
        f.write(report)

    # Save the predictions along with the true labels to a CSV file
    results_df = pd.DataFrame({
        'True_Label': test_true_labels_flat,
        'Predicted_Label': test_predictions_flat
    })
    results_df.to_csv(predictions_file_path, index=False)

    print(f"Classification report saved to {report_file_path}")
    print(f"Predictions saved to {predictions_file_path}")


In [ ]:
train_and_evaluate_cnn(
    train_file_path='/content/training_data_clean.csv',
    test_file_path='/content/test_data_clean.csv',
    report_file_path='CNN_initial_report',
    predictions_file_path='CNN_initial_predictions.csv'
)

#Hybrid CNN/RNN

In [ ]:
#As a function
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dropout, LSTM, TimeDistributed, Dense, Input
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences


def train_and_evaluate_hybrid_cnn_rnn(
    train_file_path,
    test_file_path,
    report_file_path,
    predictions_file_path,
    epochs=10,
    batch_size=32
):
    # Load the datasets
    train_data = pd.read_csv(train_file_path)
    test_data = pd.read_csv(test_file_path)

    # List of amino acids (for one-hot encoding)
    amino_acids = 'ACDEFGHIKLMNPQRSTVWY'

    # Function to one-hot encode a sequence
    def one_hot_encode(seq, aa_list):
        encoding = np.zeros((len(seq), len(aa_list)), dtype=int)
        for i, aa in enumerate(seq):
            if aa in aa_list:
                encoding[i, aa_list.index(aa)] = 1
        return encoding

    # Encode the sequences for training and testing
    train_encoded = [one_hot_encode(seq, amino_acids) for seq in train_data['seq']]
    test_encoded = [one_hot_encode(seq, amino_acids) for seq in test_data['seq']]

    # Find the maximum sequence length in the training and testing datasets
    max_seq_len = max(max(len(seq) for seq in train_data['seq']),
                      max(len(seq) for seq in test_data['seq']))

    # Pad the sequences to the maximum length
    train_sequences = pad_sequences(train_encoded, maxlen=max_seq_len, padding='post', dtype='float32')
    test_sequences = pad_sequences(test_encoded, maxlen=max_seq_len, padding='post', dtype='float32')

    # Encode the secondary structures as target labels
    sst3_mapping = {'H': 0, 'E': 1, 'C': 2}
    train_labels = pad_sequences([[sst3_mapping[ss] for ss in sst] for sst in train_data['sst3']],
                                 maxlen=max_seq_len, padding='post', value=-1)
    test_labels = pad_sequences([[sst3_mapping[ss] for ss in sst] for sst in test_data['sst3']],
                                maxlen=max_seq_len, padding='post', value=-1)

    # Mask out padding from labels by setting the mask value to 0 in the one-hot encoded output
    train_labels_categorical = np.where(train_labels[..., None] == -1, 0, to_categorical(train_labels, num_classes=3))
    test_labels_categorical = np.where(test_labels[..., None] == -1, 0, to_categorical(test_labels, num_classes=3))

    # Define the Hybrid CNN/RNN architecture
    model = Sequential([
        Input(shape=(train_sequences.shape[1], train_sequences.shape[2])),
        Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'),
        Dropout(0.2),
        Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'),
        Dropout(0.3),
        LSTM(64, return_sequences=True),
        Dropout(0.5),
        TimeDistributed(Dense(128, activation='relu')),
        TimeDistributed(Dense(3, activation='softmax')),
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(
        train_sequences,
        train_labels_categorical,
        epochs=epochs,
        batch_size=batch_size,
        validation_split=0.1,
        verbose=1
    )

    # Make predictions on the test set
    test_predictions = model.predict(test_sequences)
    test_predictions_labels = np.argmax(test_predictions, axis=-1)
    test_true_labels = np.argmax(test_labels_categorical, axis=-1)

    # Flatten the arrays to create a single list of predictions and true labels
    test_predictions_flat = test_predictions_labels.flatten()
    test_true_labels_flat = test_true_labels.flatten()

    # Generate the classification report
    report = classification_report(test_true_labels_flat, test_predictions_flat, target_names=['H', 'E', 'C'])

    # Save the classification report to a text file
    with open(report_file_path, 'w') as f:
        f.write(f"Hybrid CNN/RNN Test Accuracy: {model.evaluate(test_sequences, test_labels_categorical, verbose=0)[1]:.4f}\n\n")
        f.write("Classification Report:\n")
        f.write(report)

    # Save the predictions along with the true labels to a CSV file
    results_df = pd.DataFrame({
        'True_Label': test_true_labels_flat,
        'Predicted_Label': test_predictions_flat
    })
    results_df.to_csv(predictions_file_path, index=False)

    print(f"Classification report saved to {report_file_path}")
    print(f"Predictions saved to {predictions_file_path}")

In [ ]:
train_and_evaluate_hybrid_cnn_rnn(
    train_file_path='/content/training_data_clean.csv',
    test_file_path='/content/test_data_clean.csv',
    report_file_path='Hybrid_initial_report.txt',
    predictions_file_path='Hybrid_initial_predictions.csv'
)

#Secondary testing

#Random Forest

In [ ]:
#Test 2a
train_and_evaluate_rf(
    train_file_path='/content/training_data__part2_clean.csv',
    test_file_path='/content/test_data_part2.csv',
    report_file_path='RF_part2a_report.txt',
    predictions_file_path='RF_part2a_predictions.csv'
)

#CNN

In [ ]:
# Test 2a
train_and_evaluate_cnn(
    train_file_path='/content/training_data__part2_clean.csv',
    test_file_path='/content/test_data_part2_clean.csv',
    report_file_path='CNN_part2a_report.txt',
    predictions_file_path='CNN_part2a_predictions.csv'
)


#Hybrid CNN/RNN

In [ ]:
#Test 2a
train_and_evaluate_hybrid_cnn_rnn(
    train_file_path='/content/training_data__part2_clean.csv',
    test_file_path='/content/test_data_part2_clean.csv',
    report_file_path='Hybrid_part2a_report.txt',
    predictions_file_path='Hybrid_part2a_predictions.csv'
)